# Energy resolution

In this notebook we are going to study the energy resolution of the HGCAL prototype. As done in the last notebook, we create the energy distribution and we fit it with a gaussian shape: for a given run (let's say the 435 we are considering here), we reconstruct the energy deposit as the sum of all the rechit_energy in a given event.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf, erfc, gamma
import numpy as np
import pandas as pd
import uproot

In [ ]:
dirname = '/data_CMS/cms/vernazza/electrons/'
fname = dirname + 'ntuple_435.root'
key='rechitntupler/hits'

In [ ]:
branches = [u'rechit_chip', 'rechit_module', 'rechit_channel',
                u'rechit_energy', 'rechit_layer']

In [ ]:
ttree = uproot.open(fname)[key]
df = ttree.pandas.df(branches, entrystop=5000)

In [ ]:
df = df.reset_index(level=1,drop=True)
df.index.names = ['event']

In the last notebook we have observed how important it is to apply selections over the events. In particular we want to:

In [ ]:
# Select only the EE layers
sel = df['rechit_layer'] < 29

# Cut small energy deposits
sel &= df['rechit_energy'] > 0.5

# Remove chip 0 of layer 1 because it presented some deffects
sel &= (df['rechit_layer'] != 1) | (df['rechit_chip'] != 0)

Plot the histogram with a proper binning, then use the option `curve_fit` to fit it with a gaussian.

In [ ]:
def gaussian(x, A, sigma, mu):
    return A/(sigma * np.sqrt(2 * np.pi))*np.exp( - (x - mu)**2 / (2 * sigma**2) )

In [ ]:
popt, pcov = curve_fit(gaussian, bins_c, n, p0 = [n.max(), np.std(bins_c), np.median(bins_c)])

The fit should be performed in the interval [-1$\sigma$, 2.5$\sigma$] centered in the gaussian peak.

What is the resolution of the HGCAL? Is it independent from the energy of the the beam? What do you expect for a less energetic beam?

## Exercise

For each dataset in the `dirname` folder:
 * Open the file as a pandas `df`;
 * Create the energy sum distributions (use EE only);
 * Fit all the distributions with a gaussian around the core and extract the fit parameters;
 * Plot the resolution $\sigma/E$ from the fits vs the beam energy $E$.

In [ ]:
energies = np.array([20, 30, 50, 80, 100, 120, 150, 200, 250, 300])
runs = {
    20:  455,
    30:  596,
    50:  458,
    80:  469,
    100: 490,
    120: 620,
    150: 494,
    200: 664,
    250: 653,
    300: 435
}